# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [3]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [4]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [5]:
#clean profile data

print(profile.isnull().any())
print(profile.gender.unique())

values = {'gender': 'M', 'income': round(profile.income.mean())}

profile = profile.fillna(value=values)
profile.head()

gender               True
age                 False
id                  False
became_member_on    False
income               True
dtype: bool
[None 'F' 'M' 'O']


,gender,age,id,became_member_on,income
0,M,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,65405.0
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,M,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,65405.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,M,118,a03223e636434f42ac4c3df47e8bac43,20170804,65405.0


## Outputs- 

1. BOGO
2. discount
3. info
4. nothing

In [6]:
# combine dataFrames of profile and transcript
tempDF = pd.merge(transcript, profile, left_on='person', right_on='id')

In [7]:
tempDF.head()

,person,event,value,time,gender,age,id,became_member_on,income
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},6,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
2,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.89},132,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
3,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 17.78},144,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0


In [8]:
# get the offer id or amount and add it as a new column

txn_type = []

for val in tempDF.value:
    
    txn_type.append(list(val.values())[0])

txn_type

['9b98b8c7a33c4b65b9aebfe6a799e6d9',
 '9b98b8c7a33c4b65b9aebfe6a799e6d9',
 19.89,
 '9b98b8c7a33c4b65b9aebfe6a799e6d9',
 17.78,
 '5a8bc65990b245e5a138643cd4eb9837',
 '5a8bc65990b245e5a138643cd4eb9837',
 19.67,
 29.72,
 23.93,
 'ae264e3637204a6fb9bb56bc8210ddfd',
 'ae264e3637204a6fb9bb56bc8210ddfd',
 'f19421c1d4aa40978ebb69ca19b0e20d',
 21.72,
 'ae264e3637204a6fb9bb56bc8210ddfd',
 'f19421c1d4aa40978ebb69ca19b0e20d',
 26.56,
 'f19421c1d4aa40978ebb69ca19b0e20d',
 '0b1e1539f2cc45b7b9fa7c272da2e1d7',
 '0b1e1539f2cc45b7b9fa7c272da2e1d7',
 1.09,
 3.5,
 '3f207df678b143eea3cee63160fa8bed',
 '3f207df678b143eea3cee63160fa8bed',
 '5a8bc65990b245e5a138643cd4eb9837',
 '0b1e1539f2cc45b7b9fa7c272da2e1d7',
 '0b1e1539f2cc45b7b9fa7c272da2e1d7',
 0.06,
 '0b1e1539f2cc45b7b9fa7c272da2e1d7',
 '2906b810c7d4411798c6938adc9daaa5',
 '2906b810c7d4411798c6938adc9daaa5',
 17.88,
 21.43,
 '3f207df678b143eea3cee63160fa8bed',
 '9b98b8c7a33c4b65b9aebfe6a799e6d9',
 '9b98b8c7a33c4b65b9aebfe6a799e6d9',
 'fafdcd668e3743c1bb

In [9]:
tempDF['txn_amt_offer'] = txn_type
tempDF.head()

,person,event,value,time,gender,age,id,became_member_on,income,txn_amt_offer
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},6,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9
2,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.89},132,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,19.89
3,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 17.78},144,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,17.78


In [10]:
# combine dataFrames of tempDF and portfolio

combined_data = pd.merge(tempDF, portfolio, left_on='txn_amt_offer', right_on='id', how='left').fillna(0)

combined_data.head()

,person,event,value,time,gender,age,id_x,became_member_on,income,txn_amt_offer,reward,channels,difficulty,duration,offer_type,id_y
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"[web, email, mobile]",5.0,7.0,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},6,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"[web, email, mobile]",5.0,7.0,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
2,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.89},132,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,19.89,0.0,0,0.0,0.0,0,0
3,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"[web, email, mobile]",5.0,7.0,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 17.78},144,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,17.78,0.0,0,0.0,0.0,0,0


In [11]:
combined_data.event.unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

'offer_sent' - 1 - yes, 0 - no
,'offer_id' - offer_id -yes, else - 0
'offer_viewed' 1 - yes, 0 - no
'no_txn' - count
'offer_used' - 1- yes, 0 - no

In [12]:
transcript_sorted = transcript.sort_values('person',ignore_index=True)

data = pd.DataFrame(columns=['offer_received','offer_id','offer_viewed','no_txn','offer_used'])
j = 0 
data.loc[j,'offer_received'] = 0
data.loc[j,'offer_id'] = 0
data.loc[j,'offer_viewed'] = 0
data.loc[j,'no_txn'] = 0
data.loc[j,'offer_used'] = 0


# for i in range(len(trasncript_sorted.values)):

for i in range(100):
    
    if transcript_sorted.loc[i,'event'] == 'offer received':
        data.loc[j,'offer_received'] = 1
        data.loc[j,'offer_id'] = list(transcript_sorted.loc[i,'value'].values())[0]
        
    if transcript_sorted.loc[i,'event'] == 'offer viewed':
        data.loc[j,'offer_viewed'] = 1
        
    if transcript_sorted.loc[i,'event'] == 'transaction':
        data.loc[j,'no_txn'] = data.loc[j,'no_txn'] + 1
        
    if transcript_sorted.loc[i,'event'] == 'offer completed':
        data.loc[j,'offer_used'] = 1
    
    if (transcript_sorted.loc[i,'person'] != transcript_sorted.loc[i+1,'person']) & (i != len(transcript_sorted.values)):
        j = j + 1
        data.loc[j,'offer_received'] = 0
        data.loc[j,'offer_id'] = 0
        data.loc[j,'offer_viewed'] = 0
        data.loc[j,'no_txn'] = 0
        data.loc[j,'offer_used'] = 0
        

data.head()        

,offer_received,offer_id,offer_viewed,no_txn,offer_used
0,1,3f207df678b143eea3cee63160fa8bed,1,8,1
1,1,f19421c1d4aa40978ebb69ca19b0e20d,1,3,0
2,1,5a8bc65990b245e5a138643cd4eb9837,1,5,1
3,1,4d5c57ea9a6940dd891ad53e9dbe8da0,1,8,1
4,1,f19421c1d4aa40978ebb69ca19b0e20d,1,12,1


In [13]:
trasncript_sorted = transcript.sort_values('person',ignore_index=True)
list(trasncript_sorted.loc[1,'value'].values())[0]

8.57

In [14]:
transcript_sorted['person'].value_counts().head()


94de646f7b6041228ca7dec82adb97d2    51
8dbfa485249f409aa223a2130f40634a    49
d0a80415b84c4df4908b8403b19765e3    48
5e60c6aa3b834e44b822ea43a3efea26    48
79d9d4f86aca4bed9290350fb43817c2    48
Name: person, dtype: int64

In [17]:
temp_df = transcript_sorted[transcript_sorted['person'] == '94de646f7b6041228ca7dec82adb97d2']

temp_df.sort_values('time')

,person,event,value,time
178566,94de646f7b6041228ca7dec82adb97d2,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0
178544,94de646f7b6041228ca7dec82adb97d2,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},6
178591,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 7.41},30
178589,94de646f7b6041228ca7dec82adb97d2,offer completed,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,30
178545,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 1.47},102
178560,94de646f7b6041228ca7dec82adb97d2,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},168
178547,94de646f7b6041228ca7dec82adb97d2,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},186
178592,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 2.62},192
178590,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 0.59},204
178556,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 2.2800000000000002},246


In [16]:
len(temp_df[temp_df['event'] == 'offer completed'])

5

In [98]:
def get_rows_for_person(transaction, person):
    """
    This function is for returning rows of a perticular person
    the number of rows are the number of offers the person completed/received
    
    input - complete transcript, person of interest
    
    output - the records for the person
    
    """
    data = pd.DataFrame(columns=['person','offer_id','offer_received','offer_viewed','no_txn','amount_spent','offer_completed'])
    
    temp_df = transaction[transaction['person'] == person]
    
    # get the txn amount or offer id from each row, and drop original value column
    txn_type = []
    for val in temp_df.value:
        txn_type.append(list(val.values())[0])

    temp_df['txn_amt_offer'] = txn_type
    temp_df = temp_df.drop('value',axis=1)

    
    # get no. of txns and the amount spent
    no_txn = len(temp_df[temp_df['event'] == 'transaction'])
    total_amount = round(temp_df[temp_df['event'] == 'transaction'].txn_amt_offer.sum())
    
    
    # drop all rows with transactions as event
    temp_df = temp_df[temp_df.event != 'transaction']
    
    # check details of each offer, and add a row to output data
    for offer in temp_df.txn_amt_offer.unique():
        print('inside offer', offer)
        offer_temp_df = temp_df[temp_df.txn_amt_offer == offer]
        received = len(offer_temp_df[offer_temp_df['event'] == 'offer received'])
        viewed = len(offer_temp_df[offer_temp_df['event'] == 'offer viewed'])
        completed = len(offer_temp_df[offer_temp_df['event'] == 'offer completed'])
        print(person, offer, received, viewed, no_txn, total_amount, completed)
        print('before', data.shape)
        data_row = pd.DataFrame([[person, offer, received, viewed, no_txn, total_amount, completed]]))
        data.append(data_row)
        print('after', data.shape)
        print('after', data_row.shape)
    
    
    
    return data




In [99]:
data = get_rows_for_person(transcript_sorted,'94de646f7b6041228ca7dec82adb97d2')
data

inside offer f19421c1d4aa40978ebb69ca19b0e20d
94de646f7b6041228ca7dec82adb97d2 f19421c1d4aa40978ebb69ca19b0e20d 1 1 35 90 1
before (0, 7)
after (7, 8)
after (7,)
inside offer 9b98b8c7a33c4b65b9aebfe6a799e6d9
94de646f7b6041228ca7dec82adb97d2 9b98b8c7a33c4b65b9aebfe6a799e6d9 2 2 35 90 2
before (7, 8)
after (14, 8)
after (7,)
inside offer 4d5c57ea9a6940dd891ad53e9dbe8da0
94de646f7b6041228ca7dec82adb97d2 4d5c57ea9a6940dd891ad53e9dbe8da0 1 1 35 90 0
before (14, 8)
after (21, 8)
after (7,)
inside offer fafdcd668e3743c1bb461111dcafc2a4
94de646f7b6041228ca7dec82adb97d2 fafdcd668e3743c1bb461111dcafc2a4 1 1 35 90 1
before (21, 8)
after (28, 8)
after (7,)
inside offer 0b1e1539f2cc45b7b9fa7c272da2e1d7
94de646f7b6041228ca7dec82adb97d2 0b1e1539f2cc45b7b9fa7c272da2e1d7 1 0 35 90 1
before (28, 8)
after (35, 8)
after (7,)


C:\Users\Sriram\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,0,amount_spent,no_txn,offer_completed,offer_id,offer_received,offer_viewed,person
0,94de646f7b6041228ca7dec82adb97d2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,94de646f7b6041228ca7dec82adb97d2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
temp_df = transcript_sorted[transcript_sorted['person'] == '94de646f7b6041228ca7dec82adb97d2']
temp_df[temp_df['event'] == 'offer completed']
txn_type = []

for val in temp_df.value:
    
    txn_type.append(list(val.values())[0])


temp_df['txn_amt_offer'] = txn_type
temp_df = temp_df.drop('value',axis=1)
no_txn = len(temp_df[temp_df['event'] == 'transaction'])
print(no_txn)
total_amount = temp_df[temp_df['event'] == 'transaction'].txn_amt_offer.sum()
print(total_amount)
temp_df = temp_df[temp_df.event != 'transaction']

for offer in temp_df.txn_amt_offer.unique():
    offer_temp_df = temp_df[temp_df.txn_amt_offer == offer]
    received = len(offer_temp_df[offer_temp_df['event'] == 'offer received'])
    viewed = len(offer_temp_df[offer_temp_df['event'] == 'offer viewed'])
    completed = len(offer_temp_df[offer_temp_df['event'] == 'offer completed'])
    

35
90.23000000000002


C:\Users\Sriram\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [54]:
offer_temp_df = temp_df[temp_df.txn_amt_offer == 'f19421c1d4aa40978ebb69ca19b0e20d']
offer_temp_df

,person,event,time,txn_amt_offer
178544,94de646f7b6041228ca7dec82adb97d2,offer viewed,6,f19421c1d4aa40978ebb69ca19b0e20d
178566,94de646f7b6041228ca7dec82adb97d2,offer received,0,f19421c1d4aa40978ebb69ca19b0e20d
178589,94de646f7b6041228ca7dec82adb97d2,offer completed,30,f19421c1d4aa40978ebb69ca19b0e20d
